# Python for Data Analysis v2 | Notes_ Chapter_8 数据规整：聚合、合并和重塑

 本人以简书作者 SeanCheney 系列专题文章并结合原书为学习资源，记录个人笔记，仅作为知识记录及后期复习所用，原作者地址查看 [简书 SeanCheney](https://www.jianshu.com/u/130f76596b02)，如有错误，还望批评指教。——ZJ


>原作者：SeanCheney | [《利用Python进行数据分析·第2版》第8章 数据规整：聚合、合并和重塑](hhttps://www.jianshu.com/p/cfc035bae567) | 來源：简书

>[Github:wesm](https://github.com/wesm/pydata-book) | [Github:中文 BrambleXu](https://github.com/BrambleXu/pydata-notebook)|
简书:[利用   Python   进行数据分析·第2版](https://www.jianshu.com/c/52882df3377a)

环境：   Python    3.6 

---

# Chapter 8 数据规整：聚合、合并和重塑


在许多应用中，数据可能分散在许多文件或数据库中，存储的形式也不利于分析。本章关注可以聚合、合并、重塑数据的方法。

首先，我会介绍pandas的层次化索引，它广泛用于以上操作。然后，我深入介绍了一些特殊的数据操作。在第14章，你可以看到这些工具的多种应用。

## 8.1 层次化索引

层次化索引（hierarchical indexing）是pandas的一项重要功能，它使你能在一个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理高维度数据。我们先来看一个简单的例子：创建一个Series，并用一个由列表或数组组成的列表作为索引：

```


```

看到的结果是经过美化的带有 MultiIndex 索引的 Series 的格式。索引之间的“间隔”表示“直接使用上面的标签”：


```


```

对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集的操作更简单：


```


```
有时甚至还可以在“内层”中进行选取：


```


```
层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。例如，可以通过unstack方法将这段数据重新安排到一个DataFrame中：


```


```

unstack的逆运算是stack：

```


```

stack和unstack将在本章后面详细讲解。

对于一个DataFrame，每条轴都可以有分层索引：

```


```

各层都可以有名字（可以是字符串，也可以是别的Python对象）。如果指定了名称，它们就会显示在控制台输出中：


```


```

>注意：小心区分索引名state、color与行标签。

有了部分列索引，因此可以轻松选取列分组：


```


```

可以单独创建MultiIndex然后复用。上面那个DataFrame中的（带有分级名称）列可以这样创建：


```


```
## 重排与分级排序

有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象（但数据不会发生变化）：

```


```

而sort_index则根据单个级别中的值对数据进行排序。交换级别时，常常也会用到sort_index，这样最终结果就是按照指定顺序进行字母排序了：


```


```

而sort_index则根据单个级别中的值对数据进行排序。交换级别时，常常也会用到sort_index，这样最终结果就是按照指定顺序进行字母排序了：

```


```

## 根据级别汇总统计

许多对 DataFrame 和 Series 的描述和汇总统计都有一个 level 选项，它用于指定在某条轴上求和的级别。再以上面那个 DataFrame 为例，我们可以根据行或列上的级别来进行求和：

```


```

这其实是利用了pandas的groupby功能，本书稍后将对其进行详细讲解。

## 使用DataFrame的列进行索引

人们经常想要将DataFrame的一个或多个列当做行索引来用，或者可能希望将行索引变成DataFrame的列。以下面这个DataFrame为例：

```


```

DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame：

```


```

默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来：


```


```
reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面：


```


```

DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame：


```


```
默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来：

```


```

reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面：


```


```
## 8.2 合并数据集

pandas对象中的数据可以通过一些方式进行合并：

- pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来。SQL或其他关系型数据库的用户对此应该会比较熟悉，因为它实现的就是数据库的join操作。
- pandas.concat可以沿着一条轴将多个对象堆叠到一起。
- 实例方法combine_first可以将重复数据编接在一起，用一个对象中的值填充另一个对象中的缺失值。
我将分别对它们进行讲解，并给出一些例子。本书剩余部分的示例中将经常用到它们。

## 数据库风格的DataFrame合并

数据集的合并（merge）或连接（join）运算是通过一个或多个键将行链接起来的。这些运算是关系型数据库（基于SQL）的核心。pandas的merge函数是对数据应用这些算法的主要切入点。

以一个简单的例子开始：

```


```

这是一种多对一的合并。df1 中的数据有多个被标记为 a 和 b 的行，而 df2 中key 列的每个值则仅对应一行。对这些对象调用 merge 即可得到：

```


```

注意，我并没有指明要用哪个列进行连接。如果没有指定，merge就会将重叠列的列名当做键。不过，最好明确指定一下：

```


```
如果两个对象的列名不同，也可以分别进行指定：


```


```
可能你已经注意到了，结果里面c和d以及与之相关的数据消失了。默认情况下，merge做的是“内连接”；结果中的键是交集。其他方式还有"left"、"right"以及"outer"。外连接求取的是键的并集，组合了左连接和右连接的效果：

```


```
表8-1对这些选项进行了总结。

![表8-1 不同的连接类型](./images/8_1.png)

多对多的合并有些不直观。看下面的例子：

```


```
多对多连接产生的是行的笛卡尔积。由于左边的DataFrame有3个"b"行，右边的有2个，所以最终结果中就有6个"b"行。连接方式只影响出现在结果中的不同的键的值：

```


```
要根据多个键进行合并，传入一个由列名组成的列表即可：

```


```

结果中会出现哪些键组合取决于所选的合并方式，你可以这样来理解：多个键形成一系列元组，并将其当做单个连接键（当然，实际上并不是这么回事）。

>注意：在进行列－列连接时，DataFrame对象中的索引会被丢弃。

对于合并运算需要考虑的最后一个问题是对重复列名的处理。虽然你可以手工处理列名重叠的问题（查看前面介绍的重命名轴标签），但 merge 有一个更实用的suffixes 选项，用于指定附加到左右两个 DataFrame 对象的重叠列名上的字符串：

```


```
merge的参数请参见表8-2。使用DataFrame的行索引合并是下一节的主题。

表8-2 merge函数的参数

![](./images/8_2_0.png)
![](./images/8_2_1.png)

indicator 添加特殊的列_merge，它可以指明每个行的来源，它的值有left_only、right_only或both，根据每行的合并数据的来源。

## 索引上的合并

有时候，DataFrame中的连接键位于其索引中。在这种情况下，你可以传入left_index=True 或 right_index=True（或两个都传）以说明索引应该被用作连接键：

```


```
由于默认的merge方法是求取连接键的交集，因此你可以通过外连接的方式得到它们的并集：

```


```
对于层次化索引的数据，事情就有点复杂了，因为索引的合并默认是多键合并：


```


```



```


```




```


```




```


```





```


```


```


```




```


```



```


```



```


```



```


```



```


```




```


```




```


```





```


```


```


```




```


```



```


```



```


```



```


```



```


```




```


```




```


```





```


```


```


```




```


```



```


```



```


```



```


```



```


```




```


```




```


```





```


```


```


```




```


```



```


```



```


```



```


```



```


```




```


```




```


```